In [1]:
from IPython.core.display import HTML
HTML("\n".join(open("c360_style.css", 'r').readlines()))

## Libs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import keras
plt.style.use('ggplot')
import plotly.express as px

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')
#from google.colab import drive
#drive.mount('/content/drive')
#DATA_FILE='drive/My Drive/Proyecto/df_final_c.txt'

## Data Load

In [4]:
df7 = pd.read_csv('df_final_c.txt', sep=';')

In [8]:
def map_one_hot(df, column_names, result_column_name):
    mapper = {} 
    for i, col_name in enumerate(column_names):
        for val in df[col_name].unique():
            mapper[str(val) + str(i)] = len(mapper)
         
    df_ext = df.copy()
    
    def one_hot(values):
        v = np.zeros( len(mapper) )
        for i, val in enumerate(values): 
            v[ mapper[str(val) + str(i)] ] = 1
        return v    
    
    df_ext[result_column_name] = df_ext[column_names].values.tolist()
    df_ext[result_column_name] = df_ext[result_column_name].map(one_hot)
    
    return df_ext

In [9]:
df8 = map_one_hot(df7, ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat8'], 'cats')
df9 = map_one_hot(df8, ['campaign'], 'campaigns').sort_values(by=['timestamp_norm'])

## Optimization

### Optimization technique - 1

In [12]:
coste_por_campaña = df9.groupby(['campaign']).agg({'cost':'sum', 'timestamp': 'count', 'conversion':'sum'})
coste_por_campaña = coste_por_campaña.reset_index()
coste_por_campaña.columns = ['campaña', 'coste', 'filas', 'conversiones']
coste_por_campaña = coste_por_campaña.sort_values(by=['filas'], ascending= False)
coste_por_campaña.head()

,campaña,coste,filas,conversiones
47,31772643,7.642684,17435,8443
13,6886817,4.791087,6105,3144
7,5544859,1.441321,4747,3839
12,6686704,4.163911,4056,1883
36,25419531,0.848049,3312,469


In [14]:
coste_por_campaña.tail()

,campaña,coste,filas,conversiones
19,11042365,0.021770,60,18
46,31748229,0.040551,55,54
27,18496278,0.003356,49,2
0,73322,0.012472,47,18
9,6341185,0.005944,33,2


In [15]:
import rpy2.robjects as robjects

data = robjects.r("""
pasta = 1000
distribucion = runif(50)
pasta_por_campaña = pasta*distribucion/sum(distribucion)
""")

distribucion = list(robjects.r["distribucion"])
pasta_por_campaña = list(robjects.r["pasta_por_campaña"])
print(distribucion)
print(pasta_por_campaña)

[0.6596177536994219, 0.6442346326075494, 0.4988163379020989, 0.3068592941854149, 0.25210025487467647, 0.4802203585859388, 0.8540772320702672, 0.1306388673838228, 0.8614637507125735, 0.12758055608719587, 0.5854923785664141, 0.5424733189865947, 0.3129093167372048, 0.6269915355369449, 0.6881075010169297, 0.03305716044269502, 0.5336406549904495, 0.3303628384601325, 0.9231261680833995, 0.4706958383321762, 0.9373952995520085, 0.5041466907132417, 0.4711658665910363, 0.5750361951068044, 0.2175043656025082, 0.8994515081867576, 0.3922942702192813, 0.3895811690017581, 0.41203446546569467, 0.14078866364434361, 0.19617789052426815, 0.09109307359904051, 0.46728894859552383, 0.7780575414653867, 0.307125338120386, 0.5004679700359702, 0.1161363918799907, 0.18422651756554842, 0.5866152783855796, 0.7934530852362514, 0.7479456719011068, 0.5382001539692283, 0.812983849318698, 0.1298699437174946, 0.7684523642528802, 0.6046150110196322, 0.9897594612557441, 0.6432638932019472, 0.7311148215085268, 0.5687565335

#### Simulation

In [17]:
conversiones_totales = 0
for i in range(0, coste_por_campaña.shape[0]):
    filas_para_esta_campaña = df9[df9['campaign'] == coste_por_campaña['campaña'][i]]
    filas_para_esta_campaña = filas_para_esta_campaña[['timestamp', 'campaign', 'cost', 'conversion']]
    pasta_para_esta_campaña = pasta_por_campaña[i]

    filas_para_esta_campaña['coste_acumulado'] = filas_para_esta_campaña['cost'].cumsum()
    filas_para_esta_campaña['se_puede_pagar'] = filas_para_esta_campaña['coste_acumulado'] < pasta_para_esta_campaña
    filas_para_esta_campaña['cuanto_he_convertido'] = filas_para_esta_campaña['conversion'] * filas_para_esta_campaña['se_puede_pagar']

    conversiones = filas_para_esta_campaña['cuanto_he_convertido'].sum()
    conversiones_totales = conversiones_totales + conversiones

    if(i <10):
    print("Campaña "+ str(i) + " - pasta "+ str(round(pasta_para_esta_campaña, 5)) + " - conversiones " + str(conversiones))


Campaña 0 - pasta 26.01276 - conversiones 18
Campaña 1 - pasta 25.40611 - conversiones 467
Campaña 2 - pasta 19.67138 - conversiones 273
Campaña 3 - pasta 12.10134 - conversiones 985
Campaña 4 - pasta 9.94185 - conversiones 24
Campaña 5 - pasta 18.93802 - conversiones 10
Campaña 6 - pasta 33.68149 - conversiones 348
Campaña 7 - pasta 5.15189 - conversiones 3839
Campaña 8 - pasta 33.97278 - conversiones 88
Campaña 9 - pasta 5.03128 - conversiones 2


### Optimization technique - 2

#### Data preprocessing for modelling

In [18]:
def features_for_modelling(df):

    def pairwise_max(series):
        return np.max(series.tolist(), axis = 0).tolist()
      
    def scalar (df):
        min_max_scaler = MinMaxScaler()
        for cname in ('click', 'cost', 'conversion'):
        x = df[cname].values.reshape(-1, 1) 
        df[cname] = min_max_scaler.fit_transform(x)    
        return df
    
    aggregation = {
        'campaigns': pairwise_max,
        'cats': pairwise_max,
        'click': 'sum',
        'cost': 'sum',
        'conversion': 'max'
    }
    
    df_agg = df.groupby(['jid']).agg(aggregation)
    df_agg = scalar(df_agg)
    
    df_agg['features'] = df_agg[['campaigns', 'cats', 'click', 'cost']].values.tolist()
    
    return (
        np.stack(df_agg['features'].map(lambda x: np.hstack(x)).values),
        df_agg['conversion'].values
    )

In [19]:
x, y = features_for_modelling(df9)
print(np.shape(x))

(16657, 295)


In [20]:
from sklearn.model_selection import train_test_split
n_campaigns = 50
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.20, random_state = 1)

#### Logistic Regression - Keras

In [21]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from keras.constraints import NonNeg

m = np.shape(x)[1]
    
model = Sequential()  
model.add(Dense(1, input_dim=m, activation='sigmoid', name = 'contributions')) 

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy']) 
history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_val, y_val)) 
score = model.evaluate(x_test, y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Epoch 1/10
84/84 [==============================] - 1s 4ms/step - loss: 0.6804 - accuracy: 0.5574 - val_loss: 0.6368 - val_accuracy: 0.6537
Epoch 2/10
84/84 [==============================] - 0s 2ms/step - loss: 0.6187 - accuracy: 0.6815 - val_loss: 0.5982 - val_accuracy: 0.6919
Epoch 3/10
84/84 [==============================] - 0s 2ms/step - loss: 0.5844 - accuracy: 0.7149 - val_loss: 0.5726 - val_accuracy: 0.7167
Epoch 4/10
84/84 [==============================] - 0s 2ms/step - loss: 0.5503 - accuracy: 0.7426 - val_loss: 0.5549 - val_accuracy: 0.7272
Epoch 5/10
84/84 [==============================] - 0s 2ms/step - loss: 0.5483 - accuracy: 0.7454 - val_loss: 0.5426 - val_accuracy: 0.7366
Epoch 6/10
84/84 [==============================] - 0s 2ms/step - loss: 0.5281 - accuracy: 0.7563 - val_loss: 0.5338 - val_accuracy: 0.7426
Epoch 7/10
84/84 [==============================] - 0s 2ms/step - loss: 0.5213 - accuracy: 0.7544 - val_loss: 0.5272 - val_accuracy: 0.7471
Epoch 8/10
84/84 [==

In [22]:
from sklearn.utils.extmath import softmax

keras_logreg = model.get_layer('contributions').get_weights()[0].flatten()[0:n_campaigns]
keras_logreg = softmax([keras_logreg]).flatten()

#### Logistic Regression - Scikit Learn

In [23]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(solver = "saga")
logisticRegr.fit(x_train, y_train)
score = logisticRegr.score(x_test, y_test)

#### LTA

In [24]:
def last_touch_attribution(df):
    
    def count_by_campaign(df):
        counters = np.zeros(n_campaigns)
        for campaign_one_hot in df['campaigns'].values:
            campaign_id = np.argmax(campaign_one_hot)
            counters[campaign_id] = counters[campaign_id] + 1
        return counters
        
    campaign_impressions = count_by_campaign(df)
    
    df_converted = df[df['conversion'] == 1]
    idx = df_converted.groupby(['jid'])['timestamp_norm'].transform(max) == df_converted['timestamp_norm']
    campaign_conversions = count_by_campaign(df_converted[idx])
        
    return campaign_conversions / campaign_impressions
    
lta = last_touch_attribution(df9)

#### Simulation

In [25]:
def get_campaign_id(x_journey_step):
    return np.argmax(x_journey_step[0:n_campaigns])

In [30]:
def simulate_budget_roi(df, budget_total, attribution, verbose=False):
    budgets = np.ceil(attribution * (budget_total / np.sum(attribution)))
    
    if(verbose):
        print(budgets)
    
    blacklist = set()
    conversions = set()
    for i in range(df.shape[0]):
        campaign_id = get_campaign_id(df.loc[i]['campaigns']) 
        jid = df.loc[i]['jid']
        if jid not in blacklist:
            if budgets[campaign_id] >= 1:
                budgets[campaign_id] = budgets[campaign_id] - 1
                if(df.loc[i]['conversion'] == 1):
                    conversions.add(jid)
            else:
                blacklist.add(jid)
        
        if(verbose):
            if(i % 10000 == 0):
                print('{:.2%} : {:.2%} budget spent'.format(i/df.shape[0], 1.0 - np.sum(budgets)/budget_total ))
        
        if(np.sum(budgets) < budget_total * 0.02):
            break
            
    return len(conversions.difference(blacklist)), conversions, blacklist

In [27]:
pitches = [0.1]
attributions = [lta]

for i, pitch in enumerate(pitches):
    for j, attribution in enumerate(attributions):
        reward = simulate_budget_roi(df9, 10000, attribution**pitch)
        print('{} {} : {}'.format(pitch, j, reward))

0.1 0 : 460


In [39]:
pitches = [0.1, 0.25, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
attributions = [lta, keras_logreg]
attributions_str = ['lta', 'keras_logreg']
conversiones = []
blacklists = []
rewards = []
id_pitch = []
id_attributions = []

for i, pitch in enumerate(pitches):
    for j, attribution in enumerate(attributions):
        reward, conv, black = simulate_budget_roi(df9, 10000, attribution**pitch)
        id_pitch.append(pitch)
        id_attributions.append(attributions_str[j])
        rewards.append('reward')
        conversiones.append(conv)
        blacklists.append(black)
        print('{} {} : {}'.format(pitch, j, reward))

0.1 0 : 460
0.1 1 : 438
0.25 0 : 473
0.25 1 : 450
0.5 0 : 495
0.5 1 : 466
1.0 0 : 542
1.0 1 : 481
1.5 0 : 587
1.5 1 : 513
2.0 0 : 645
2.0 1 : 571
2.5 0 : 708
2.5 1 : 576
3.0 0 : 765
3.0 1 : 571
